In [1]:
import sys

# Replace this with the path to the repository
sys.path.append("/home/zhenhao/ETFMH/")


In [2]:
from db_sketching.kmer_selection import KMerInformationGain, KMerChiSquaredTest, KMerVariance
from db_sketching.kmer_set import KMerSet, FracMinHash


def cond(kmer_hash):
    hash = (976369 * kmer_hash + 1982627) % 10000
    if hash < 100:
        return True
    else:
        return False

kmer_set = FracMinHash(cond, 12, True)
learner = KMerChiSquaredTest(kmer_set)

In [3]:
import glob
from pathlib import Path

categories = glob.glob("../data_temp/*")

for category in categories:
    name = Path(category).stem
    for file in glob.glob(category + "/*.fna"):
        learner.insert_reference(file, name)
        print(file)

../data_temp/Anaerovoracaceae/GCA_017523605.1.fna
../data_temp/Anaerovoracaceae/GCA_944386615.1.fna
../data_temp/Anaerovoracaceae/GCA_012511395.1.fna
../data_temp/Anaerovoracaceae/GCA_017505405.1.fna
../data_temp/Anaerovoracaceae/GCA_934281585.1.fna
../data_temp/Anaerovoracaceae/GCA_000488855.1.fna
../data_temp/Anaerovoracaceae/GCA_943192965.1.fna
../data_temp/Anaerovoracaceae/GCA_017511535.1.fna
../data_temp/Anaerovoracaceae/GCA_905206455.1.fna
../data_temp/Anaerovoracaceae/GCA_015058145.1.fna
../data_temp/Bacteroidaceae/GCA_022740765.1.fna
../data_temp/Bacteroidaceae/GCA_015549665.1.fna
../data_temp/Bacteroidaceae/GCA_022721415.1.fna
../data_temp/Bacteroidaceae/GCA_000598945.1.fna
../data_temp/Bacteroidaceae/GCA_007896595.1.fna
../data_temp/Bacteroidaceae/GCA_014871205.1.fna
../data_temp/Bacteroidaceae/GCA_934196725.1.fna
../data_temp/Bacteroidaceae/GCA_021410705.1.fna
../data_temp/Bacteroidaceae/GCA_902755065.1.fna
../data_temp/Bacteroidaceae/GCA_902791395.1.fna
../data_temp/Enterob

In [5]:
res = learner.select_kmers(threshold=1)

In [6]:
res

{2097153: 5.48257150789766e-09,
 3670020: 7.882583474838611e-15,
 13369348: 6.70064004282267e-12,
 2883592: 0.0,
 8912909: 6.500265858910836e-08,
 9699348: 5.555556015224283e-13,
 20: 3.25998450456666e-10,
 4456470: 6.148303059450022e-06,
 5242909: 3.458757724672523e-10,
 31: 2.2726686340623914e-06,
 6029348: 5.321428853122256e-10,
 786470: 3.25998450456666e-10,
 1572909: 4.9201843888679875e-05,
 12845104: 0.0,
 12058676: 7.742151364453775e-11,
 2359348: 1.981648178883688e-11,
 8388665: 1.3984317743798158e-06,
 7602237: 4.746270043654022e-11,
 9175104: 1.155742168634788e-13,
 3932226: 6.117761852664216e-10,
 8388676: 3.430035144802446e-10,
 3145798: 1.9181253074762594e-06,
 4718665: 2.949514610328663e-08,
 3932237: 7.331024676204834e-12,
 15204432: 2.0801587785612696e-08,
 5505104: 1.2051726283601738e-10,
 262226: 9.483784202402035e-09,
 4718676: 4.041215358463468e-06,
 1048665: 1.3160528222755374e-10,
 262237: 1.2777446767309186e-08,
 1835104: 3.4387201974794834e-09,
 11534432: 1.4584

In [20]:
import seaborn
import numpy as np

(np.array(list(res.values())) > 0.001).sum()

21234

In [14]:
len(res.values())

83550

In [16]:
# Test if the informative k-mers give better classification results.

10

In [21]:
informative_kmers = set([kmer for kmer in res if res[kmer] < 0.001])

In [22]:
def is_informative(kmer_hash):
    return kmer_hash in informative_kmers

In [18]:
len(informative_kmers)

71577

In [21]:
len(res)

167366

In [24]:
import orthoani
from Bio.SeqIO import read, parse

categories = glob.glob("../data_temp/*")

y = []
pred_FMH = []
pred_inf = []
pred_ANI = []

for i in categories:
    for j in categories:
        #if i == j:
        #    continue
        # Compare two genomes in the classes
        for genomes_i in glob.glob(i + "/*.fna"):
            for genomes_j in glob.glob(j + "/*.fna"):
                print("Comparing", genomes_i, genomes_j)
                # FracMinHash
                f_i = FracMinHash(cond, 12, True)
                f_j = FracMinHash(cond, 12, True)
                f_i.insert_file(genomes_i)
                f_j.insert_file(genomes_j)
                pred_FMH.append(f_i.resemblence(f_j))
                print(pred_FMH[-1])

                # Informative k-mers
                f_i = FracMinHash(is_informative, 12, True)
                f_j = FracMinHash(is_informative, 12, True)
                f_i.insert_file(genomes_i)
                f_j.insert_file(genomes_j)
                pred_inf.append(f_i.resemblence(f_j))
                print(pred_inf[-1])
                
                # OrthoANI
                try:
                    genome_1_read = parse(genomes_i,"fasta")
                    genome_2_read = parse(genomes_j,"fasta")
                    ortho_ani_value = orthoani.orthoani(genome_1_read,genome_2_read)

                    pred_ANI.append(ortho_ani_value)
                except:
                    pred_ANI.append(0)
                print(pred_ANI[-1])
                # True label
                if i == j:
                    y.append(1)
                else:
                    y.append(0)
                
                print(y[-1], pred_FMH[-1], pred_inf[-1], pred_ANI[-1])
        
        print(i, j)
    




Comparing ../data_temp/Anaerovoracaceae/GCA_017523605.1.fna ../data_temp/Anaerovoracaceae/GCA_017523605.1.fna
1.0
1.0
0.9995340528634361
1 1.0 1.0 0.9995340528634361
Comparing ../data_temp/Anaerovoracaceae/GCA_017523605.1.fna ../data_temp/Anaerovoracaceae/GCA_944386615.1.fna
0.12182396606574761
0.1288914906072056
0.6429113366336634
1 0.12182396606574761 0.1288914906072056 0.6429113366336634
Comparing ../data_temp/Anaerovoracaceae/GCA_017523605.1.fna ../data_temp/Anaerovoracaceae/GCA_012511395.1.fna
0.09367836584993124
0.09802907915993538
0.6243216129032259
1 0.09367836584993124 0.09802907915993538 0.6243216129032259
Comparing ../data_temp/Anaerovoracaceae/GCA_017523605.1.fna ../data_temp/Anaerovoracaceae/GCA_017505405.1.fna
0.15757527983603972
0.16676697763065504
0.6540957299270073
1 0.15757527983603972 0.16676697763065504 0.6540957299270073
Comparing ../data_temp/Anaerovoracaceae/GCA_017523605.1.fna ../data_temp/Anaerovoracaceae/GCA_934281585.1.fna
0.1232902292143224
0.132947642464564

KeyboardInterrupt: 